Functions are defined with the def statement. The order and number of arguments must match those given in the function definition. If a mismatch exists, a TypeError exception is raised. When a function defines a parameter with a default value, that parameter and all the parameters that follow are optional. If values are not assigned to all the optional parame- ters in the function definition, a SyntaxError exception is raised.

Default parameter values are always set to the objects that were supplied as values when the function was defined. 

In [2]:
a = 10
def foo(x=a):
    return x

a = 5 # Reassign 'a'.
foo()

10

In addition, the use of mutable objects as default values may lead to unintended behavior

In [3]:
def foo(x, items=[]):
    items.append(x)
    return items

foo(1)
foo(2)
foo(3)

[1, 2, 3]

Notice how the default argument retains modifications made from previous invocations. To prevent this, it is better to use None and add a check as follows:

In [4]:
def foo(x, items=None):
    if items is None:
        items = []
    items.append(x)
    return items

A function can accept a variable number of parameters if an asterisk (*) is added to the last parameter name. In this case, all the remaining arguments are placed into the args variable as a tuple. To pass a tuple args to a function as if they were parameters, the *args syntax can be used in a function call.

If the last argument of a function definition begins with \**, all the additional keyword arguments (those that don’t match any of the other parameter names) are placed in a dictionary and passed to the function.This can be a useful way to write functions that accept a large number of potentially open-ended configuration options that would be too unwieldy to list as parameters.

In [ ]:
def fprintf(file, fmt, *args):
    file.write(fmt % args)

# Use fprintf. args gets (42,"hello world", 3.45) 
fprintf(out,"%d %s %f", 42, "hello world", 3.45)

Function arguments can also be supplied by explicitly naming each parameter and spec- ifying a value.These are known as keyword arguments.

def foo(w,x,y,z):
    statements

Keyword argument invocation 

foo(x=3, y=22, w='hello', z=[1,2])

With keyword arguments, the order of the parameters doesn’t matter. However, unless there are default values, you must explicitly name all of the required function parame- ters. If you omit any of the required parameters or if the name of a keyword doesn’t match any of the parameter names in the function definition, a TypeError exception is raised. Also, since any Python function can be called using the keyword calling style, it is generally a good idea to define functions with descriptive argument names. Positional arguments and keyword arguments can appear in the same function call, provided that all the positional arguments appear first, values are provided for all non- optional arguments, and no argument value is defined more than once. Here’s an example:

foo('hello', 3, z=[1,2], y=22)  
foo(3, 22, w='hello', z=[1,2]) # TypeError. Multiple values for w

In [7]:
# Accept variable number of positional or keyword arguments 
def spam(*args, **kwargs):
    pass
# args is a tuple of positional args # kwargs is dictionary of keyword args

When a function is invoked, the function parameters are simply names that refer to the passed input objects.The underlying semantics of parameter passing doesn’t neatly fit into any single style, such as “pass by value” or “pass by reference,” that you might know about from other programming languages. For example, if you pass an immutable value, the argument effectively looks like it was passed by value. However, if a mutable object (such as a list or dictionary) is passed to a function where it’s then modified, those changes will be reflected in the original object.

In [8]:
a = [1, 2, 3, 4, 5] 
def square(items):
    for i,x in enumerate(items):
        items[i] = x * x # Modify items in-place

square(a) # Changes a to [1, 4, 9, 16, 25]
print(a)

[1, 4, 9, 16, 25]


Functions that mutate their input values or change the state of other parts of the pro- gram behind the scenes like this are said to have side effects. As a general rule, this is a programming style that is best avoided because such functions can become a source of subtle programming errors as programs grow in size and complexity (for example, it’s not obvious from reading a function call if a function has side effects). Such functions interact poorly with programs involving threads and concurrency because side effects typically need to be protected by locks.

The return statement returns a value from a function. If no value is specified or you omit the return statement, the None object is returned.To return multiple values, place them in a tuple.

# Scoping

Each time a function executes, a new local namespace is created.This namespace repre- sents a local environment that contains the names of the function parameters, as well as the names of variables that are assigned inside the function body.When resolving names, the interpreter first searches the local namespace. If no match exists, it searches the glob- al namespace.The global namespace for a function is always the module in which the function was defined. If the interpreter finds no match in the global namespace, it makes a final check in the built-in namespace. If this fails, a NameError exception is raised.

Inside a function, you can explicitly make a variable global by declaring it so before the variable is used, using the global statement.

In [9]:
def fun():
    global a
    a = 1
    b = 2
    
a = "one"
b = "two"
fun()
print(a,b)

1 two


Python 3 introduced the **nonlocal** keyword that allows you to assign to variables in an outer, but non-global, scope. An example will illustrate what I mean.

In [10]:
def outside():
    msg = "Outside!"
    def inside():
        msg = "Inside!"
        print(msg)
    inside()
    print(msg)
    
outside()

Inside!
Outside!


In [11]:
def outside():
    msg = "Outside!"
    def inside():
        nonlocal msg
        msg = "Inside!"
        print(msg)
    inside()
    print(msg)
outside()

Inside!
Inside!


# Functions As Objects And Closures

Functions are first-class objects in Python.This means that they can be passed as argu- ments to other functions, placed in data structures, and returned by a function as a result.

In [12]:
def callf(func):
    func()

def helloworld():
    print("Hello world!")

callf(helloworld)

Hello world!


When a function is handled as data, it implicitly carries information related to the sur- rounding environment where the function was defined.This affects how free variables in the function are bound.

In [15]:
# foo.py
x = 42
def callf(func):
    return func()

In [ ]:
import foo
x = 37

def helloworld():
    print(x)

foo.callf(helloworld)
#Output - 37

In this example, notice how the function helloworld() uses the value of x that’s defined in the same environment as where helloworld() was defined. Thus, even though there is also an x defined in foo.py and that’s where helloworld() is actually being called, that value of x is not the one that’s used when helloworld() executes. When the statements that make up a function are packaged together with the envi- ronment in which they execute, the resulting object is known as a closure.The behavior of the previous example is explained by the fact that all functions have a __globals__ attribute that points to the global namespace in which the function was defined.This always corresponds to the enclosing module in which a function was defined.

When nested functions are used, closures capture the entire environment needed for the inner function to execute.

In [ ]:
import foo 
def bar():
    x = 13
    def helloworld():
        print(x)
        
    foo.callf(helloworld)
##Output - 13

Closures and nested functions are especially useful if you want to write code based on the concept of lazy or delayed evaluation.

In [17]:
from urllib.request import urlopen
def page(url):
    def get():
        return urlopen(url).read()
    return get

In this example, the page() function doesn’t actually carry out any interesting compu- tation. Instead, it merely creates and returns a function get() that will fetch the con- tents of a web page when it is called.Thus, the computation carried out in get() is actually delayed until some later point in a program when get() is evaluated. 

In [24]:
python = page("http://www.python.org")
jython = page("http://www.jython.org")

In [25]:
python

<function __main__.page.<locals>.get()>

In [31]:
jython

<function __main__.page.<locals>.get()>

In [32]:
#pydata = python()
#jydata = jython()

In this example, the two variables python and jython are actually two different ver- sions of the get() function. Even though the page() function that created these values is no longer executing, both get() functions implicitly carry the values of the outer variables that were defined when the get() function was created. Thus, when get() executes, it calls urlopen(url) with the value of url that was originally supplied to page(). With a little inspection, you can view the contents of variables that are carried along in a closure.

In [33]:
python.__closure__

(<cell at 0x10a124618: str object at 0x10a695d68>,)

In [34]:
python.__closure__[0].cell_contents

'http://www.python.org'

A closure can be a highly efficient way to preserve state across a series of function calls. For example, consider this code that runs a simple counter

In [36]:
def countdown(n): 
    def next():
        nonlocal n
        r=n
        n -= 1
        return r
    return next

# Example use
next = countdown(10)
while True:
    v = next()
    if not v:
        break
    else:
        print(v)

10
9
8
7
6
5
4
3
2
1


In this code, a closure is being used to store the internal counter value n.The inner function next() updates and returns the previous value of this counter variable each time it is called. Programmers not familiar with closures might be inclined to imple- ment similar functionality using a class such as this:

In [41]:
class Countdown(object):
    def __init__(self,n):
        self.n = n
    def next(self):
        r = self.n
        self.n -= 1
        return r
    
# Example use
c = Countdown(10) 
while True:
    v = c.next()
    if not v:
        break
    else:
        print(v)

10
9
8
7
6
5
4
3
2
1


However, if you increase the starting value of the countdown and perform a simple timing benchmark, you will find that that the version using closures runs much faster (almost a 50% speedup). The fact that closures capture the environment of inner functions also make them useful for applications where you want to wrap existing functions in order to add extra capabilities.

# Decorators

A decorator is a function whose primary purpose is to wrap another function or class. The primary purpose of this wrapping is to transparently alter or enhance the behavior of the object being wrapped.

In [ ]:
@trace
def square(x):
    return x*x

The above is a short hand for the following

In [ ]:
def square(x):
    return x*x
square = trace(square)

In the example, a function square() is defined. However, immediately after its defini- tion, the function object itself is passed to the function trace(), which returns an object that replaces the original square.

In [70]:
def decorate(func):
    print("Decorating function: " + func.__name__)
    def wrapper_func(*args):
        print("Executing function: " + func.__name__)
        return func(*args)
    return wrapper_func

@decorate
def myfunction(param):
    print(param)
    
myfunction("hello world")

Decorating function: myfunction
Executing function: myfunction
hello world


# Generators & Coroutines

If a function uses the yield keyword, it defines an object known as a generator. A gener- ator is a function that produces a sequence of values for use in iteration. Here’s an example:

In [46]:
def countdown(n):
    print("Counting down from %d" % n)
    while n > 0:
        yield n
        n -= 1
    return

If you call this function, you will find that none of its code starts executing. Instead, a generator object is returned.The generator object, in turn, executes the func- tion whenever next() is called (or __next__() in Python 3)

In [47]:
k = countdown(10)

In [48]:
k

<generator object countdown at 0x10a6b2930>

In [50]:
k.__next__()

Counting down from 10


10

In [51]:
k.__next__()

9

In [52]:
k.__next__()

8

When next() is invoked, the generator function executes statements until it reaches a yield statement. The yield statement produces a result at which point execution of the function stops until next() is invoked again. Execution then resumes with the statement following yield. You normally don’t call next() directly on a generator but use it with the for statement, sum(), or some other operation that consumes a sequence.

In [55]:
for n in countdown(10):
    pass
    #statements

Counting down from 10


In [56]:
a = sum(countdown(10))

Counting down from 10


In [57]:
a

55

A generator function signals completion by returning or raising StopIteration, at which point iteration stops. It is never legal for a generator to return a value other than None upon completion. A subtle problem with generators concerns the case where a generator function is only partially consumed. For example, consider this code:

In [58]:
for n in countdown(10):
    if n == 2: 
        break

Counting down from 10


In this example, the for loop aborts by calling break, and the associated generator never runs to full completion.To handle this case, generator objects have a method close() that is used to signal a shutdown.When a generator is no longer used or deleted, close() is called. Normally it is not necessary to call close(), but you can also call it manually

In [60]:
def countdown(n):
    print("Counting down from %d" % n)
    try:
        while n > 0: 
            yield n
            n=n-1
    except GeneratorExit:
        print("Only made it to %d" % n)

Although it is possible to catch GeneratorExit, it is illegal for a generator function to handle the exception and produce another output value using yield. Moreover, if a program is currently iterating on generator, you should not call close() asynchronous- ly on that generator from a separate thread of execution or from a signal handler.

Inside a function, the yield statement can also be used as an expression that appears on the right side of an assignment operator.

In [61]:
def receiver():
    print("Ready to receive")
    while True:
        n = (yield)
        print("Got %s" % n)

In [63]:
r = receiver()

In [64]:
r.__next__()

Ready to receive


In [65]:
r.send(3)

Got 3


In [66]:
r.send("Aditya")

Got Aditya


In [67]:
r.close()

In [68]:
r.send(8)

StopIteration: 

In this example, the initial call to next() is necessary so that the coroutine executes statements leading to the first yield expression. At this point, the coroutine suspends, waiting for a value to be sent to it using the send() method of the associated generator object r.The value passed to send() is returned by the (yield) expression in the coroutine. Upon receiving a value, a coroutine executes statements until the next yield statement is encountered.

The requirement of first calling next() on a coroutine is easily overlooked and a common source of errors.Therefore, it is recommended that coroutines be wrapped with a decorator that automatically takes care of this step.

In [69]:
def coroutine(func):
    def start(*args,**kwargs):
        g = func(*args,**kwargs)
        g.next()
        return g
    return start

@coroutine
def coroutinefunc():
    pass